In [1]:
from beir import util
from beir.datasets.data_loader import GenericDataLoader


/home/victor/Documents/travail_epita/venv_epita/lib/python3.10/site-packages/beir/util.py:2: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [ ]:
from load_dataset import create_smaller_corpus, download_dataset

is_clean_corpus = False
number_trash = 100000
corpus, queries, qrels = download_dataset("./data")
if is_clean_corpus:
	corpus = create_smaller_corpus(corpus, queries, qrels, "./data", number_trash)


In [5]:
print("size of corpus is", len(corpus))
print("size of queries is", len(queries))
print("size of qrels is", len(qrels))

size of corpus is 140724
size of queries is 400
size of qrels is 400


We have 3 values.
-Corpus is a dictionary representing the description of each DBPedia page, representing an event, a person, a culture of anything else. Each element element contains the title of a page and the text associated.
-Queries represent a series of request by a user. It could be a question, a name, or a sentence describing what he looking for.
-Qrels represent the possible answers that match with queries. Each values is marked between 0 and 2. The best they have the best the response is.